# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [39]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [40]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [41]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [59]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_unit_mean = sfo_data.groupby(['year'])['housing_units'].mean()
    housing_units_per_year_output = housing_unit_mean.hvplot(
    x='year',
    y='housing_units',
    kind='bar',
    color='black',
    #groupby='neighborhood',
    xlabel='Year',
    ylabel='Housing Units per Year',
    ylim=(370000,387500)    
)
    return housing_units_per_year_output

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    average_housing_cost_sfo = sfo_data.groupby(['year'])['sale_price_sqr_foot','gross_rent'].mean()
    average_gross_rent_output = average_housing_cost_sfo.hvplot(
    x='year',
    y='gross_rent',
    kind='line',
    color='blue',
    #groupby='neighborhood',
    xlabel='Year',
    ylabel='Average Rental prices'
)
    return average_gross_rent_output


def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_housing_cost_sfo = sfo_data.groupby(['year'])['sale_price_sqr_foot','gross_rent'].mean()
    average_sales_price_output = average_housing_cost_sfo.hvplot(
    x='year',
    y='sale_price_sqr_foot',
    kind='line',
    color='red',
    #groupby='neighborhood',
    xlabel='Year',
    ylabel='Average Price Per Square Foot'
)
    return average_sales_price_output


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    avg_prices_nbrhd = sfo_data.groupby(['year','neighborhood'])['sale_price_sqr_foot','gross_rent'].mean().reset_index()
    avg_prices_nbrhd1 = avg_prices_nbrhd[['year','neighborhood','sale_price_sqr_foot']].set_index('year')#.sort_values(by='sale_price_sqr_foot')
    average_price_by_neighborhood_output = avg_prices_nbrhd1.hvplot(
    x='year',
    y='sale_price_sqr_foot',
    kind='line',
    groupby='neighborhood',
    xlabel='Year',
    ylabel='Average Price Per Square Foot'
)
    return average_price_by_neighborhood_output


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    most_expensive = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()
    most_expensive_10 = most_expensive.sort_values('sale_price_sqr_foot', ascending=False)[0:10]
    top_most_expensive_neighborhoods_output = most_expensive_10.hvplot(
    x='neighborhood',
    y='sale_price_sqr_foot',
    kind='bar',
    xlabel='Year',
    ylabel='Average Sale Price per Square Foot',
    color = 'Crimson',
    rot=90
)
    return top_most_expensive_neighborhoods_output

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    avg_prices_nbrhd = sfo_data.groupby(['year','neighborhood'])['sale_price_sqr_foot','gross_rent'].mean().reset_index()
    most_expensive_neighborhoods_rent_sales_output = avg_prices_nbrhd.hvplot(
    y='gross_rent',
    x='year',
    kind='bar',
    groupby='neighborhood',
    ylabel='Num Housing Units',
    label="Average Monthly Rent"
) * avg_prices_nbrhd.hvplot(
    y='sale_price_sqr_foot',
    x='year',
    kind='bar',
    groupby='neighborhood',
    ylabel='Num Housing Units',
    label="Average Price Per Square Foot"
)

    return most_expensive_neighborhoods_rent_sales_output


    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    most_expensive = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()
    most_expensive_10 = most_expensive.sort_values('sale_price_sqr_foot', ascending=False)[0:10]

    df_expensive_neighborhoods_per_year = most_expensive[most_expensive["neighborhood"].isin(most_expensive_10["neighborhood"])].reset_index().dropna()
    parallel_coordinates_output = px.parallel_categories(
    df_expensive_neighborhoods_per_year,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    )
    return parallel_coordinates_output


def parallel_categories():
    """Parallel Categories Plot."""
    most_expensive = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()
    most_expensive_10 = most_expensive.sort_values('sale_price_sqr_foot', ascending=False)[0:10]

    df_expensive_neighborhoods_per_year = most_expensive[most_expensive["neighborhood"].isin(most_expensive_10["neighborhood"])].reset_index().dropna()
    parallel_categories_output = px.parallel_coordinates(df_expensive_neighborhoods_per_year[['sale_price_sqr_foot','housing_units','gross_rent']], color = 'sale_price_sqr_foot')
    return parallel_categories_output


def neighborhood_map():
    """Neighborhood Map."""
    file_path = Path("Data/neighborhoods_coordinates.csv")
    nbrhd_coord = pd.read_csv(file_path, index_col="Neighborhood")
    nbrhd_mean = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()
    nbrhd_mean = nbrhd_mean.rename(columns={"neighborhood": "Neighborhood"})
    nbrhd_mean = nbrhd_mean.set_index('Neighborhood')
    
    nbrhd_mean_coord = pd.concat([nbrhd_coord,nbrhd_mean], axis="columns", join="inner").reset_index()
    map_box_api = os.getenv("MAPBOX_API_KEY")
    px.set_mapbox_access_token(map_box_api)

# Create a scatter mapbox to analyze neighborhood info
    neighborhood_map_output = px.scatter_mapbox(
    nbrhd_mean_coord,
    lat = "Lat",
    lon = "Lon",
    color = "gross_rent",
    size = 'sale_price_sqr_foot',
    zoom = 10,
    title = "Average Sale Price Per Square Foot and Gross Rent in San Francisco"
)
    return neighborhood_map_output


def sunburst():
    """Sunburst Plot."""
    most_expensive = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()
    most_expensive_10 = most_expensive.sort_values('sale_price_sqr_foot', ascending=False)[0:10]
    df_expensive_neighborhoods = sfo_data[sfo_data["neighborhood"].isin(most_expensive_10["neighborhood"])].reset_index().dropna()
    sunburst_output = px.sunburst(
    df_expensive_neighborhoods,
    path=['year', 'neighborhood'], values='sale_price_sqr_foot',
                  color='gross_rent', 
    #height = int(10),
    width= int(50),
     title = "Cost Analysis of Most Expensive neighborhoods in San Francisco per Year"
    #hover_data=['iso_alpha'],
                  #color_continuous_scale='RdBu',
                  #color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
)
    return sunburst_output

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [61]:
# Create a Title for the Dashboard
#row = pn.Row(scatter_plot, bar_plot)
#row

welcome_column = pn.Column(
    '# Real Estate Analysis of San Francisco 2010 to 2016',
    '## Welcome to Dashboard',
    '### This is a collection of dashboards to highlight the average rental prices and sales prices per square foot in the neighborhood of San Francisco. You can navigate through the different tabs to see the interative graphs to access the analysis.'
    neighborhood_map()
)

housing_unit_column = pn.Row(
    '## San Francisco housing units per year',
    housing_units_per_year()
)

average_cost_sf = pn.Column(
    '## San Francisco averages',
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood()
)

expensive_price_by_neighborhood = pn.Column(  
    '## San Francisco''s expensive neighborhoods',
    top_most_expensive_neighborhoods(),
    most_expensive_neighborhoods_rent_sales()
)

parallels_columns = pn.Row(  
    '## San Francisco in Parallel analysis plots',
    parallel_categories(),
    parallel_coordinates()
)

sunburst_column = pn.Row(
    '## San Francisco in Sunburst plot',
    sunburst()
)

# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Welcome", welcome_column),
    ("Housing Per Year in San Francisco",housing_unit_column),
    ("Parallel Categories and Coordinates", parallels_columns),
    ("Average Cost in San Francisco", average_cost_sf),
    ("Expensive Price by Neighborhood", expensive_price_by_neighborhood),
    ("Sunburst", sunburst_column)
)


# Create the dashboard
tabs

C:\Users\Jay Cao\Anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:133: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\Jay Cao\Anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:20: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\Jay Cao\Anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:36: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\Jay Cao\Anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:52: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\Jay Cao\Anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:68: FutureWarning:

Indexing with multiple keys (implicitly converted 

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Plotly(Figure)
    [1] Row
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [2] Row
        [0] Plotly(Figure, visible=False)
        [1] Plotly(Figure, visible=False)
    [3] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [5] Row
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)

## Serve the Panel Dashboard

In [63]:
# Serve the# dashboard
tabs.servable()

Tabs(active=5)
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Plotly(Figure, relayout_data={'autosize': True}, visible=False)
    [1] Row
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [2] Row
        [0] Plotly(Figure, relayout_data={'autosize': True}, visible=False)
        [1] Plotly(Figure, relayout_data={'autosize': True}, visible=False)
    [3] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [5] Row
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [45]:
# housing_units_per_year()

In [46]:
# average_gross_rent()

In [47]:
# average_sales_price()

In [48]:
# average_price_by_neighborhood()

In [49]:
# top_most_expensive_neighborhoods()

In [50]:
# most_expensive_neighborhoods_rent_sales()

In [51]:
# neighborhood_map()#.show()

In [52]:
# parallel_categories()

In [53]:
 # parallel_coordinates()

In [54]:
# sunburst()